<a href="https://colab.research.google.com/github/0yacinek0/nlp/blob/practice/%5Bgithub_nlp%5D_5_%EB%8B%A4%EC%A4%91_%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 레이블의 갯수가 두개 이상인 데이터의 분류를 해보자. 그런데 한글 데이터셋을 못구해서... 일단 영어 데이터로 진행하겠다. 이 경우는 6개의 클래스가 있는 TREC 데이터를 분석할 것이며, 따라서 LABEL 필드에서 dtype을 선언할 필요가 없다.

### Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd # 현재 경로 확인 

'/content'

In [ ]:
%cd /content/drive/MyDrive/논문/sentimentP/dataset

/content/drive/MyDrive/논문/sentimentP/dataset


In [ ]:
ls

Mecab-ko-for-Google-Colab/  ratings_train.txt  train_data.csv
ratings_test.txt            test_data.csv


### 필요한 패키지와 랜덤시드 설정

In [ ]:
# !pip install torchtext==0.10.1 
!pip install torch==1.8.0 torchtext==0.9.0
"""
아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.

torchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?
pip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.

https://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils

"""

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.5/735.5 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 62.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.8.0 which is incompatible.


'\n아마 예전 코드를 실행하시면서 지금은 사라진 객체를 생성하시려다가 생기는 오류 같습니다.\n\ntorchtext 버전을 0.10.x 이나 그 이전으로 낮춰보시는 것은 어떠실까요?\npip install torchtext==0.10.1 과 같은 식으로 버전을 지정하셔서 설치하실 수 있습니다.\n\nhttps://stackoverflow.com/questions/73055161/importerror-cannot-import-name-unicode-csv-reader-from-torchtext-utils\n\n'

# 전처리
* 레이블의 갯수가 두개 이상인 데이터의 분류를 해보자. 그런데 한글 데이터셋을 못구해서... 일단 영어 데이터로 진행하겠다. 이 경우는 6개의 클래스가 있는 TREC 데이터를 분석할 것이며, 따라서 LABEL 필드에서 dtype을 선언할 필요가 없다.

In [ ]:
# 코랩 내 다운로드 https://wikidocs.net/94600
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

fatal: destination path 'Mecab-ko-for-Google-Colab' already exists and is not an empty directory.
/content/drive/MyDrive/논문/sentimentP/dataset/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 34.0 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-29 17:38:48--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.2, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.ta

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
!pip install torchtext.datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torchtext.datasets (from versions: none)
ERROR: No matching distribution found for torchtext.datasets


In [ ]:
import torch
import torchtext
from torchtext import data
from torchtext import datasets
import random

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.determinisitic = True

In [ ]:
TEXT = torchtext.legacy.data.Field(tokenize='spacy')
LABEL = torchtext.legacy.data.Field()

In [ ]:
train_data, test_data = datasets.TREC.splits(TEXT, LABEL, fine_grained=False)
# train_data, test_data = torchtext.legacy.datasets.TREC(root='.', split=('train', 'test'))
# https://huggingface.co/datasets/trec


AttributeError: ignored

### TREC 데이터 불러오기 에러 
* AttributeError: module 'torchtext.datasets' has no attribute 'TREC'

In [ ]:
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [ ]:
len(train_data), len(valid_data), len(test_data)

### 데이터 살펴보기

In [ ]:
vars(train_data[-3])

In [ ]:
vars(train_data[-2])

### 단어장 만들기. 
*  훈련 데이터 갯수가 많지 않아서 최대 갯수 설정은 할 필요가 없긴 하나 함

In [ ]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, 
                max_size = MAX_VOCAB_SIZE,
                vectors = 'glove.6B.100d',
                unk_init = torch.Tensor.normal_
                )
LABEL.build_vocab(train_data)

### 데이터 라벨 타입
* 데이터의 라벨은 다음과 같은 6개이다.

* HUM for questions about humans
* ENTY for questions about entities
* DESC for questions asking you for a description
* NUM for questions where the answer is numerical
* LOC for questions where the answer is a location
* ABBR for questions asking about abbreviations

In [ ]:
print(LABEL.vocab.stoi)

### 이터레이터 설정

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
next(iter(train_iterator)).text.shape

### CNN 모델 사용

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size = (fs, embedding_dim))
                                    for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #text = [sent_len, batch_size]
        text = text.permute(1,0)
        #text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #embedded = [batch_size, 1, sent_len, emb_dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conv_n = [batch_size, n_filters, sent_len - fs[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch_size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #cat = [batch_size, n_filters * len(filter_size)]
        
        return self.fc(cat)

### 하이퍼파라미터 설정. 데이터셋 사이즈가 작으므로 필터 사이즈를 작게 설정

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

In [ ]:
model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

### 파라미터 개수 세어보기

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'이 모델은 {count_parameters(model):,}개의 훈련 가능한 파라미터가 있습니다.')

### 사전 훈련된 단어 벡터 적용

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

### unk_token과 pad_token의 벡터를 0으로 만들기

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

### 이 모델은 기존처럼 하나의 확률값을 출력으로 하는 것이 아니라 각 클래스별 확률을 출력하므로, 손실 함수를 기존 BCEWithLogitsLoss에서 CrossEntropyLoss로 수정해야한다.

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

### 각 클래스별 스코어를 출력으로 받으면 가장 높은 점수를 가지는 클래스를 반환하도록 하는 정확도 함수를 만들자. 다만 배치로 작업하고 있으므로, 배치의 평균으로 구하자.

In [ ]:
def categorical_accuracy(preds, y):
    """배치별 정확도를 출력하는 함수"""
    max_preds = preds.argmax(dim=1, keepdim=True)
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])    

In [ ]:
pred = torch.tensor([[5.0,0.1,0.3],[2.3,3.1,0.7]])
pred

In [ ]:
pred.argmax(dim=1).shape, pred.argmax(dim=1, keepdim=True).shape

In [ ]:
pred.argmax(dim=1, keepdim=True).squeeze(1).eq(torch.tensor([1,1]))

### 훈련함수 만들기

In [ ]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### eval 함수도 비슷함

In [ ]:
def eval(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = eval(model, train_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc*100:.2f}%')

### 테스트셋에 검증

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = eval(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

### 임의의 문장에 대해 판정해보기

In [ ]:
import spacy
nlp = spacy.load('en')

def predict_class(model, sentence, min_len=4):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim=1)
    return max_preds.item()    

### 실제로 확인해보기

In [ ]:
q = 'The one who told me the truth is Inhwan Lee.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')

In [ ]:
q = 'The central part of Korea is called Gyung-gi do.'
pred_class = predict_class(model, q)
print(f'Predicted class is: {pred_class} = {LABEL.vocab.itos[pred_class]}')